In [1]:
import os
#os.chdir('..')
input_folder = "Input"
output_folder = "Output"
basepath = os.getcwd()
input_path = os.path.join(basepath, input_folder)
output_path = os.path.join(basepath, output_folder)
image_input = input_path + '\\'
image_output = output_path + '\\'
print(basepath)
print(input_path)

C:\Users\chunk\Jupyter Projects\colorizeBlackandWhite
C:\Users\chunk\Jupyter Projects\colorizeBlackandWhite\Input


In [2]:
os.chdir(basepath)
os.chdir('DeOldify')

In [3]:
#HIDE
import io
import PIL
from PIL import Image
from deoldify import device
from deoldify.device_id import DeviceId
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import requests
import torch
import shutil

device.set(device=DeviceId.GPU0)
if not torch.cuda.is_available():
    print('GPU not available.')

In [4]:
#HIDE

import fastai
from deoldify.visualize import *
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

In [5]:
#HIDE

colorizer = get_image_colorizer(artistic=True)

In [16]:
#HIDE
grand = widgets.IntSlider(min=15,max=39)

url_pic = widgets.Text(placeholder='Please put url here: ')

file = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)


button = widgets.Button(description='Colorize',
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
    )


uploader = widgets.FileUpload(
    accept='image/*',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
)


restoreBox = widgets.Checkbox(
    value=False,
    description='Restore',
    disabled=False,
    indent=False
)


def clear_input_folder():
    os.chdir(input_path)
    file_list = os.listdir()
    for image in file_list:
        try:
            os.remove(image)
        except:
            os.rmdir(image)
    os.chdir(basepath)

In [17]:
# DISPLAY BUTTON  

out = widgets.Output()

def file_upload():
    os.chdir(input_path)
    for name, file_info in uploader.value.items():
        picture = Image.open(io.BytesIO(file_info['content']))
        picture = picture.save(name)
    os.chdir(basepath)
    
    
def image_colorise():
    os.chdir(input_path)
    for name, file_info in uploader.value.items():
        picture = Image.open(io.BytesIO(file_info['content']))
        picture = picture.save(name)
    images = os.listdir()
    os.chdir(output_path)
    for image in images:
        p = image_input + image
        os.chdir(basepath+'\\DeOldify')
        image_path = colorizer.plot_transformed_image(path=p, render_factor=grand.value, compare=True, watermarked=False)
        show_image_in_notebook(image_path)
        picture = Image.open(image_path)
        os.chdir(output_path)
        picture = picture.save('COLORIZED_' + image)
    clear_input_folder()
    os.chdir(basepath)
    
    
    
def get_image_from_url():
    os.chdir(input_path)
    with open('URL_image.jpg', 'wb') as handle:
        response = requests.get(url_pic.value, stream=True)

        if not response.ok:
            print(response)

        for block in response.iter_content(1024):
            if not block:
                break

            handle.write(block)
    os.chdir(basepath)
    
    
    
def restore():
    os.chdir(basepath)
    upload_path = input_path
    upload_output_path = input_path
    
    os.chdir(input_path)
    uploaded = os.listdir()
    os.chdir(basepath)
    os.chdir('color_restoration')
    
    for filename in uploaded:
        shutil.move(os.path.join(basepath, filename), os.path.join(upload_path, filename))
    
    !python run.py --input_folder ..\Input --output_folder ..\Output --GPU 0 --with_scratch --HR
    os.chdir(basepath)
    
def import_detection():
    if url_pic.value != '':
        get_image_from_url()
        image_colorise()
    elif uploader.value.items() is not None:
        file_upload()
        image_colorise()
    else:
        print('No images found.')

def restore_detection(event):
    if restoreBox.value is True:
        restore()
        import_detection()
    else:
        import_detection()
        
button.on_click(restore_detection)
text = widgets.HTML(value="Please provide text to colorize picture: ")
left_box = widgets.VBox([text, url_pic])
ortext = widgets.HTML(value="-OR-")
uploadtext = widgets.HTML(value="Upload Image File:")
second_row = widgets.VBox([ortext, uploadtext, uploader])
renderText = widgets.HTML(value="Please select the render factor: ")
third_row = widgets.VBox([renderText, grand])
restoreText = widgets.HTML(value="Restore Image: (May cause undesirable effect)")
fourth_row = widgets.VBox([restoreText, restoreBox])
vbox_result = widgets.VBox([left_box, second_row, third_row, fourth_row, button, out])
vbox_result


# linking button and function together using a button's method
# displaying button and its output together